<a href="https://colab.research.google.com/github/RicardColomOrero/easymoney/blob/eva/Easymoney_grupo5_1006.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ENUNCIADO CAPSTONE

 
## 1.- PowerBI (Opcional)
Hemos decidido reorientar nuestra estrategia comercial hacia el
**aumento de la penetración de la cartera actual de productos en nuestro mercado**, con la idea de obtener una
mejor rentabilidad de nuestra base actual de clientes (lo del cuadrante barato de la matriz de Ansoff). 

A cambio, reduciremos la actividad intensiva de captación que veníamos realizando desde que arrancamos. En este sentido, y para acabar de definir la estrategia, estaría genial poder tener algunos análisis que nos ayuden
a tomar decisiones. Como te decía, ahora tenemos un autoservicio, pero esta foto simple no nos ayuda demasiado. 
Si ni siquiera sé cuantos productos hemos vendido este mes o si son los clientes nuevos quiénes contratan o los que ya teníamos!

Sería genial incluso que pudieses utilizar el reciente producto adquirido
PowerBI para **preparar unos KPI’s de lo que tenemos en la empresa**

## 2.- Propensión (modelo de propensión a compra)

Una vez que hayamos hecho el **análisis de nuestra BBDD nos gustaría poder vender más y mejor nuestros productos**, sé que los data scientist tienen una “bola de cristal” y saben qué clientes son más propensos a
comprar nuestros productos así que te pido por favor hagas un **modelo de propensión a compra**, se que tenemos muchos productos, así que te **,dejo a tu criterio científico qué productos elegir, o pueden ser varios,**, ya
me sorprenderás

## 3.- Segmentación

Una vez obtenido la propensión de los productos sería genial poder conocer mejor a nuestros clientes para orientar nuestra actividad comercial. Hace tiempo vengo teniendo la idea de que podríamos **segmentar la base
de clientes en 7 u 8 grupos de clientes similares**... sería genial** identificarlos, y poder saber cómo son los clientes
que pertenecen a estos grupos**... nos ayudaría muchísimo a la hora de definir todo nuestro plan de marketing de cara a futuro!

## 4.- Caso de uso

Perfecto! Tenemos las segmentaciones de clientes, tenemos las propensiones y ahora lo que nos falta es realizar
**un caso de uso que nos permita estimar en base a una campaña X cuánto dinero podemos ganar**, 

¿A cuántos clientes tenemos que impactar?

¿Qué productos ofrecer?
 
¿Impactamos a todos lo segmentos indicados?


# Importación de librerías

In [2]:
import numpy as np # Arrays
import pandas as pd # Series and Dataframes

from google.colab import files

#visualizacion
import matplotlib.pyplot as plt 
import seaborn as sns 

pd.options.display.max_rows = 200
plt.style.use('ggplot') # Beautify Plots 

#Modelling
from sklearn import model_selection
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
import graphviz


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importación dataset

In [4]:
file_dir = "/content/drive/MyDrive/TFM  Data Science/datasets_TFM_y_diccionario/"
#file_dir = "/content/drive/MyDrive/Master_DS/TFM_Data_Science/datasets_TFM_y_diccionario/" #Culebra file_dir

In [5]:
file_sales ="sales.csv"
file_product_description ="product_description.csv"
file_customer_sociodemographics ="customer_sociodemographics.csv"
file_customer_commercial_activity = "customer_commercial_activity.csv"
file_customer_products ="customer_products.csv"

In [6]:
sales_raw = pd.read_csv(file_dir + file_sales)
product_description_raw = pd.read_csv(file_dir +file_product_description)
customer_sociodemographics_raw = pd.read_csv(file_dir + file_customer_sociodemographics)
customer_commercial_activity_raw = pd.read_csv(file_dir + file_customer_commercial_activity)
customer_products_raw = pd.read_csv(file_dir + file_customer_products)

# 2.- DATA UNDERSTANDING

## 2.1.- Análisis univariante de datos

### Sales

* pk_sale: Identificador de venta - (primary key)
* cid: Identificador de cliente - (foreing key)
* month_sale: Mes de venta. Mes en el que se contrata un producto.
* product_ID: Identificador de producto  - (foreing key)
* net_margin: Margen neto generado por la venta ($)

In [7]:
# Análisis inicial de sales

sales_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240773 entries, 0 to 240772
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  240773 non-null  int64  
 1   pk_sale     240773 non-null  int64  
 2   cid         240773 non-null  int64  
 3   month_sale  240773 non-null  object 
 4   product_ID  240773 non-null  int64  
 5   net_margin  240773 non-null  float64
dtypes: float64(1), int64(4), object(1)
memory usage: 11.0+ MB


In [8]:
sales_raw.head()

,Unnamed: 0,pk_sale,cid,month_sale,product_ID,net_margin
0,0,6666,33620,2018-05-01,2335,952.9
1,1,6667,35063,2018-06-01,2335,1625.2
2,2,6668,37299,2018-02-01,2335,1279.7
3,3,6669,39997,2018-02-01,2335,1511.9
4,4,6670,44012,2018-02-01,2335,1680.3


In [9]:
sales_raw.describe(include = ['object'])

,month_sale
count,240773
unique,16
top,2018-10-01
freq,27693


In [10]:
sales_raw.describe(exclude = ['object'])

,Unnamed: 0,pk_sale,cid,product_ID,net_margin
count,240773.000000,240773.000000,2.407730e+05,240773.000000,240773.000000
mean,120386.000000,127052.000000,1.363934e+06,3945.408202,606.878164
std,69505.322519,69505.322519,1.751721e+05,1632.821700,1735.806089
min,0.000000,6666.000000,1.589100e+04,1119.000000,25.000000
25%,60193.000000,66859.000000,1.301012e+06,2312.000000,61.400000
50%,120386.000000,127052.000000,1.409253e+06,4657.000000,69.100000
75%,180579.000000,187245.000000,1.475320e+06,4657.000000,79.200000
max,240772.000000,247438.000000,1.553571e+06,9001.000000,18466.400000


In [11]:
sales_raw.isnull().sum()

Unnamed: 0    0
pk_sale       0
cid           0
month_sale    0
product_ID    0
net_margin    0
dtype: int64

In [12]:
sales_raw[sales_raw.duplicated()]

,Unnamed: 0,pk_sale,cid,month_sale,product_ID,net_margin


In [13]:
# ACCIONES A LLEVAR A CABO 

# Eliminar columna Unnamed
# Renombrar columnas para alinear el naming, por ejemplo variable cid como "client_ID", pk_sale como "sale_ID"
# Cambiar los identificadores (product_ID, customer_ID) a variable categórica
# Cambiar tipo de month_sale a fecha
# Disgregar la fecha en mes, año... (otros: trimestre, fin de semana, festivo...)


### Product Description

* pk_product_ID	= Identificador de producto - (primary key)
* product_desc	= Descripción del producto
* family_product	= Vertical del producto. Grupo al que pertenece.

In [14]:
# Análisis inicial de product description

product_description_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      13 non-null     int64 
 1   pk_product_ID   13 non-null     int64 
 2   product_desc    13 non-null     object
 3   family_product  13 non-null     object
dtypes: int64(2), object(2)
memory usage: 544.0+ bytes


In [15]:
product_description_raw

,Unnamed: 0,pk_product_ID,product_desc,family_product
0,0,4657,em_acount,account
1,1,3819,debit_card,payment_card
2,2,1364,pension_plan,pension_plan
3,3,2234,payroll,account
4,4,2235,payroll_account,account
5,5,8871,emc_account,account
6,6,2312,credit_card,payment_card
7,7,2335,short_term_deposit,investment
8,8,2336,long_term_deposit,investment
9,9,2673,securities,investment


In [16]:
product_description_raw.describe(include = ['object'])

,product_desc,family_product
count,13,13
unique,13,5
top,em_acount,account
freq,1,4


In [17]:
product_description_raw.describe(exclude = ['object'])

,Unnamed: 0,pk_product_ID
count,13.00000,13.000000
mean,6.00000,3614.538462
std,3.89444,2564.251438
min,0.00000,1119.000000
25%,3.00000,2235.000000
50%,6.00000,2336.000000
75%,9.00000,4033.000000
max,12.00000,9001.000000


In [18]:
product_description_raw.isnull().sum()

Unnamed: 0        0
pk_product_ID     0
product_desc      0
family_product    0
dtype: int64

In [19]:
product_description_raw[product_description_raw.duplicated()]

,Unnamed: 0,pk_product_ID,product_desc,family_product


In [20]:
# ACCIONES A LLEVAR A CABO 

# Eliminar columna Unnamed

### Customer Sociodemographics

* pk_cid =	Identificador de cliente
* pk_partition =	Identificador de Partición. Mensualmente se ingesta en la tabla una partición con el estado de la base de clientes.
* deceased =	Indicador de fallecimiento
* gender =	Sexo del cliente
* region_code =	Provincia de residencia del cliente (para ES)
* salary =	Ingresos brutos de la unidad familiar
* country_id =	País de residencia del cliente
* age =	Edad del cliente

In [21]:
customer_sociodemographics_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 9 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Unnamed: 0    int64  
 1   pk_cid        int64  
 2   pk_partition  object 
 3   country_id    object 
 4   region_code   float64
 5   gender        object 
 6   age           int64  
 7   deceased      object 
 8   salary        float64
dtypes: float64(2), int64(3), object(4)
memory usage: 409.4+ MB


In [22]:
customer_sociodemographics_raw.head()

,Unnamed: 0,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary
0,0,1375586,2018-01,ES,29.0,H,35,N,87218.10
1,1,1050611,2018-01,ES,13.0,V,23,N,35548.74
2,2,1050612,2018-01,ES,13.0,V,23,N,122179.11
3,3,1050613,2018-01,ES,50.0,H,22,N,119775.54
4,4,1050614,2018-01,ES,50.0,V,23,N,NaN


In [23]:
customer_sociodemographics_raw.describe(include = ['object'])

,pk_partition,country_id,gender,deceased
count,5962924,5962924,5962899,5962924
unique,17,41,2,2
top,2019-05,ES,H,N
freq,442995,5960672,3087502,5961849


In [24]:
customer_sociodemographics_raw.describe(exclude = ['object'])

,Unnamed: 0,pk_cid,region_code,age,salary
count,5.962924e+06,5.962924e+06,5.960660e+06,5.962924e+06,4.421820e+06
mean,7.155425e+06,1.234930e+06,2.581440e+01,2.976042e+01,1.155833e+05
std,3.920784e+06,1.623020e+05,1.359566e+01,1.198038e+01,2.000066e+05
min,0.000000e+00,1.589100e+04,1.000000e+00,2.000000e+00,1.202730e+03
25%,4.240386e+06,1.112532e+06,1.300000e+01,2.200000e+01,6.141532e+04
50%,7.499346e+06,1.231097e+06,2.800000e+01,2.500000e+01,8.844147e+04
75%,1.020142e+07,1.352339e+06,3.600000e+01,3.400000e+01,1.313092e+05
max,1.364731e+07,1.553689e+06,5.200000e+01,1.050000e+02,2.889440e+07


In [25]:
customer_sociodemographics_raw.isnull().sum()

Unnamed: 0            0
pk_cid                0
pk_partition          0
country_id            0
region_code        2264
gender               25
age                   0
deceased              0
salary          1541104
dtype: int64

In [26]:
customer_sociodemographics_raw["country_id"].value_counts()

ES    5960672
GB        441
FR        225
DE        199
US        195
CH        194
BR         87
BE         81
VE         79
IE         68
MX         58
AT         51
AR         51
PL         49
IT         45
MA         34
CL         30
CN         28
CA         22
LU         17
ET         17
QA         17
CI         17
SA         17
CM         17
SN         17
MR         17
NO         17
RU         17
CO         17
GA         17
GT         17
DO         17
SE         16
DJ         11
PT         11
JM         11
RO          9
HU          8
DZ          7
PE          4
Name: country_id, dtype: int64

In [27]:
# ACCIONES CUSTOMER SOCIODEMOGRAPHICS

# Gestionar nulos: (Culebra: tenemos modelo de clasificación así que lo que haremos será ponerles valores extremos)
#     region_code
#     gender
#     salary
# gender > Transformar a 1 y 0
# deceased > Transformar a 1 y 0
# country_id > Agrupar ES, OTHERS



### Customer Commercial Activity

* pk_cid = Identificador de cliente
* pk_partition =	Identificador de Partición. Mensualmente se ingesta en la tabla una partición con el estado de la base de clientes.
* entry_channel =	Canal de captación del cliente
* entry_date =	Fecha en la que realizó la primera contratación a través de easyMoney
* segment =	Segmento comercial del cliente
* active_customer =	Indicador de actividad del cliente en la app EasyMoney

In [28]:
customer_commercial_activity_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 7 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Unnamed: 0       int64  
 1   pk_cid           int64  
 2   pk_partition     object 
 3   entry_date       object 
 4   entry_channel    object 
 5   active_customer  float64
 6   segment          object 
dtypes: float64(1), int64(2), object(4)
memory usage: 318.5+ MB


In [29]:
customer_commercial_activity_raw.head()

,Unnamed: 0,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment
0,0,1375586,2018-01,2018-01,KHL,1.0,02 - PARTICULARES
1,1,1050611,2018-01,2015-08,KHE,0.0,03 - UNIVERSITARIO
2,2,1050612,2018-01,2015-08,KHE,0.0,03 - UNIVERSITARIO
3,3,1050613,2018-01,2015-08,KHD,0.0,03 - UNIVERSITARIO
4,4,1050614,2018-01,2015-08,KHE,1.0,03 - UNIVERSITARIO


In [30]:
customer_commercial_activity_raw.describe(include = ['object'])

,pk_partition,entry_date,entry_channel,segment
count,5962924,5962924,5829891,5828980
unique,17,53,68,3
top,2019-05,2016-10,KHE,03 - UNIVERSITARIO
freq,442995,382473,3113947,3900166


In [31]:
customer_commercial_activity_raw.describe(exclude = ['object'])

,Unnamed: 0,pk_cid,active_customer
count,5.962924e+06,5.962924e+06,5.962924e+06
mean,7.155425e+06,1.234930e+06,4.033845e-01
std,3.920784e+06,1.623020e+05,4.905767e-01
min,0.000000e+00,1.589100e+04,0.000000e+00
25%,4.240386e+06,1.112532e+06,0.000000e+00
50%,7.499346e+06,1.231097e+06,0.000000e+00
75%,1.020142e+07,1.352339e+06,1.000000e+00
max,1.364731e+07,1.553689e+06,1.000000e+00


In [32]:
customer_commercial_activity_raw.isnull().sum()

Unnamed: 0              0
pk_cid                  0
pk_partition            0
entry_date              0
entry_channel      133033
active_customer         0
segment            133944
dtype: int64

In [33]:
customer_commercial_activity_raw["segment"].value_counts()

03 - UNIVERSITARIO    3900166
02 - PARTICULARES     1830875
01 - TOP                97939
Name: segment, dtype: int64

In [34]:
customer_commercial_activity_raw["entry_channel"].value_counts()

KHE    3113947
KFC     890620
KHQ     590280
KAT     416084
KHK     230197
KHM     176591
KHN     108434
KFA      79020
KHD      75899
RED      60601
KHL      41736
KHF      20657
KHO       8247
KAZ       5635
KHC       5241
KBG       1662
KEH        728
KHP        691
007        613
013        397
KAG        335
KAS        305
KBZ        198
KAA        193
KAR        157
KAY        149
KAF        123
KAB         95
KFD         91
KCC         89
KAW         65
KAQ         52
KDH         51
KAE         48
KCH         48
KCB         47
KAD         45
KAM         43
KAH         40
KAK         34
KAJ         33
KDT         23
KFF         22
KBO         21
KEY         20
KGC         17
004         17
KFS         17
KCL         17
KBE         17
KBH         17
KFK         17
KGN         17
KCI         17
KGX         17
KDR         16
KFL         13
KBW         12
KBU         11
KAI         11
KCK         11
KBY         11
KES         11
KEJ          8
KHS          5
KDA          2
KFP       

In [35]:
# ACCIONES CUSTOMER COMMERCIAL ACTIVITY**


# Eliminar columna Unnamed
# Cambiar los identificadores (pk_cid) a variable categórica
# Cambiar tipo de active_customer a boolean
# Establecer primary key como index?
# Gestionar nulos: 
#     entry_channel      
#     segment            
# Cambiar tipo de entry_date a fecha
# Disgregar entry_date en mes, año, día, día de la semana (otros: trimestre, fin de semana, festivo...)
# entry_channel > OneHotEncoding o FrecuencyEncoding ( agrupar 11-12 etiquetas + OTHERS)
# segment > OneHotEncoding

### Customer Products

- pk_cid =	Identificador de cliente
- pk_partition =	Identificador de Partición. Mensualmente se ingesta en la tabla una partición con el estado de la base de clientes.
- credit_card =	Tenencia de tarjetas de crédito
- debit_card =	Tenencia de tarjetas de débito
- em_account_p =	Tenencia de Cuenta easyMoney+
- em_account_pp =	Tenencia de Cuenta easyMoney++
- em_acount =	Tenencia de Cuenta easyMoney
- emc_account =	Tenencia de Cuenta easyMoney Crypto
- funds =	Tenencia de Fondos de inversión
- loans =	Tenencia de Préstamos
- long_term_deposit =	Tenencia de Depósitos a largo plazo
- mortgage =	Tenencia de Mortgage
- payroll =	Tenencia de Domiciliaciones 
- payroll_account =	Tenencia de Cuenta bonificada por domiciliaciones
- pension_plan =	Tenencia de Plan de pensiones
- securities =	Tenencia de Valores
- short_term_deposit =	Tenencia de Short-term deposits

In [36]:
customer_products_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 18 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Unnamed: 0          int64  
 1   pk_cid              int64  
 2   pk_partition        object 
 3   short_term_deposit  int64  
 4   loans               int64  
 5   mortgage            int64  
 6   funds               int64  
 7   securities          int64  
 8   long_term_deposit   int64  
 9   em_account_pp       int64  
 10  credit_card         int64  
 11  payroll             float64
 12  pension_plan        float64
 13  payroll_account     int64  
 14  emc_account         int64  
 15  debit_card          int64  
 16  em_account_p        int64  
 17  em_acount           int64  
dtypes: float64(2), int64(15), object(1)
memory usage: 818.9+ MB


In [37]:
customer_products_raw.head()

,Unnamed: 0,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,0,1375586,2018-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
1,1,1050611,2018-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
2,2,1050612,2018-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
3,3,1050613,2018-01,1,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0
4,4,1050614,2018-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1


In [38]:
customer_products_raw.describe(include = ['object'])

,pk_partition
count,5962924
unique,17
top,2019-05
freq,442995


In [39]:
customer_products_raw.describe(exclude = ['object'])

,Unnamed: 0,pk_cid,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
count,5.962924e+06,5.962924e+06,5.962924e+06,5.962924e+06,5.962924e+06,5.962924e+06,5.962924e+06,5.962924e+06,5962924.0,5.962924e+06,5.962863e+06,5.962863e+06,5.962924e+06,5.962924e+06,5.962924e+06,5.962924e+06,5.962924e+06
mean,7.155425e+06,1.234930e+06,2.581619e-03,7.848498e-05,5.433576e-05,3.370662e-03,3.711770e-03,1.688098e-02,0.0,1.188444e-02,3.451144e-02,3.652641e-02,5.524471e-02,5.483233e-02,9.441140e-02,5.701901e-06,7.348076e-01
std,3.920784e+06,1.623020e+05,5.074401e-02,8.858828e-03,7.371080e-03,5.795948e-02,6.081113e-02,1.288255e-01,0.0,1.083660e-01,1.825388e-01,1.875960e-01,2.284573e-01,2.276527e-01,2.924002e-01,2.387859e-03,4.414356e-01
min,0.000000e+00,1.589100e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.240386e+06,1.112532e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,7.499346e+06,1.231097e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
75%,1.020142e+07,1.352339e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
max,1.364731e+07,1.553689e+06,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [40]:
customer_products_raw.isnull().sum()

Unnamed: 0             0
pk_cid                 0
pk_partition           0
short_term_deposit     0
loans                  0
mortgage               0
funds                  0
securities             0
long_term_deposit      0
em_account_pp          0
credit_card            0
payroll               61
pension_plan          61
payroll_account        0
emc_account            0
debit_card             0
em_account_p           0
em_acount              0
dtype: int64

In [41]:
# ACCIONES CUSTOMER PRODUCTS

# Eliminar columna Unnamed
# Cambiar los identificadores (pk_cid) a variable categórica
# Cambiar tipo de columnas booleanas a boolean?
# Establecer primary key como index?
# Gestionar nulos: 
#     payroll                     
#     pension_plan
# Eliminar em_account_pp, no hay datos

# Cambiar tipo de payroll y pension_plan a int

### Análisis del target

In [42]:
# Para el modelo de propensión a compra (supervisado > clasificación) trataremos de averiguar la propensión a compra de ciertos productos de cada cliente
# El target puede ser:
#  - em_acount = 4.381.602  (73,48%)
#  - debit_card = 562.968 (9,44%)
#  - payroll_account = 329.420 (5,52%)
#  - pension_plan = 2.178.020 (3,65%) ** Target interesante por negocio



# Analizando el revenue de cada producto:
#  - pension_plan = media >5200€ ** Target interesante por revenue
#  - loans = media > 2000€
#  - mortgage = media > 2000€ 
#
#
# El producto "pension_plan" resulta interesante a nivel net margin, pues es el que más beneficios genera a la compañía

# Hecho el primero modelo para "pension plan", más tarde se puede volver a lanzar con otros target de interés, por ejemplo "debit_card"o "emc_account "


TARGET = "pension_plan"
TARGET2 = 'debit_card'
TARGET3 = 'emc_account'

In [43]:
for i in ['short_term_deposit', 'loans', 'mortgage', 'funds', 'securities','long_term_deposit', 'em_account_pp','credit_card', 'payroll', 'pension_plan', 'payroll_account','emc_account', 'debit_card', 'em_account_p', 'em_acount']:
    suma_productos = customer_products_raw[i].sum()
    print (f"Suma de {i} = {suma_productos} . La media es de {round((suma_productos/customer_products_raw.shape[0])*100,2)}%")
    

Suma de short_term_deposit = 15394 . La media es de 0.26%
Suma de loans = 468 . La media es de 0.01%
Suma de mortgage = 324 . La media es de 0.01%
Suma de funds = 20099 . La media es de 0.34%
Suma de securities = 22133 . La media es de 0.37%
Suma de long_term_deposit = 100660 . La media es de 1.69%
Suma de em_account_pp = 0 . La media es de 0.0%
Suma de credit_card = 70866 . La media es de 1.19%
Suma de payroll = 205787.0 . La media es de 3.45%
Suma de pension_plan = 217802.0 . La media es de 3.65%
Suma de payroll_account = 329420 . La media es de 5.52%
Suma de emc_account = 326961 . La media es de 5.48%
Suma de debit_card = 562968 . La media es de 9.44%
Suma de em_account_p = 34 . La media es de 0.0%
Suma de em_acount = 4381602 . La media es de 73.48%


### Distribución de las variables

#### Variables numéricas

In [44]:
for i in sales_raw.select_dtypes(include= ["number"]).columns:
     print(i, sales_raw[i].value_counts())

Unnamed: 0 0         1
160520    1
160508    1
160509    1
160510    1
         ..
80262     1
80263     1
80264     1
80265     1
240772    1
Name: Unnamed: 0, Length: 240773, dtype: int64
pk_sale 6666      1
167186    1
167174    1
167175    1
167176    1
         ..
86928     1
86929     1
86930     1
86931     1
247438    1
Name: pk_sale, Length: 240773, dtype: int64
cid 1387215    10
1436512     9
1380507     9
1402957     9
1402199     9
           ..
1409414     1
1409413     1
1409412     1
1409410     1
1553571     1
Name: cid, Length: 152754, dtype: int64
product_ID 4657    117435
3819     38840
1364     19369
2234     18750
2235     18110
8871     12323
2312      5206
2335      4441
2336      4227
2673      1157
1119       880
9001        25
4033        10
Name: product_ID, dtype: int64
net_margin 63.1       890
64.8       854
63.9       851
64.1       848
62.7       841
          ... 
4917.3       1
5698.9       1
5708.1       1
11009.5      1
34.5         1
Name: net_margi

In [45]:
for i in customer_products_raw.select_dtypes(include= ["number"]).columns:
  print(i, customer_products_raw[i].value_counts())

Unnamed: 0 0           1
9344701     1
9344699     1
9344698     1
9344697     1
           ..
5074084     1
5074083     1
5074082     1
5074081     1
13647308    1
Name: Unnamed: 0, Length: 5962924, dtype: int64
pk_cid 1375586    17
1328781    17
1328783    17
1328784    17
1328786    17
           ..
1521045     1
1545224     1
1470941     1
1521049     1
1550586     1
Name: pk_cid, Length: 456373, dtype: int64
short_term_deposit 0    5947530
1      15394
Name: short_term_deposit, dtype: int64
loans 0    5962456
1        468
Name: loans, dtype: int64
mortgage 0    5962600
1        324
Name: mortgage, dtype: int64
funds 0    5942825
1      20099
Name: funds, dtype: int64
securities 0    5940791
1      22133
Name: securities, dtype: int64
long_term_deposit 0    5862264
1     100660
Name: long_term_deposit, dtype: int64
em_account_pp 0    5962924
Name: em_account_pp, dtype: int64
credit_card 0    5892058
1      70866
Name: credit_card, dtype: int64
payroll 0.0    5757076
1.0     205787


In [46]:
for i in customer_sociodemographics_raw.select_dtypes(include= ["number"]).columns:
     print(i, customer_sociodemographics_raw[i].value_counts())

Unnamed: 0 0           1
9344701     1
9344699     1
9344698     1
9344697     1
           ..
5074084     1
5074083     1
5074082     1
5074081     1
13647308    1
Name: Unnamed: 0, Length: 5962924, dtype: int64
pk_cid 1375586    17
1328781    17
1328783    17
1328784    17
1328786    17
           ..
1521045     1
1545224     1
1470941     1
1521049     1
1550586     1
Name: pk_cid, Length: 456373, dtype: int64
region_code 28.0    1185757
8.0      600362
46.0     358142
30.0     296856
41.0     290447
15.0     254891
29.0     187911
36.0     175850
3.0      171358
11.0     160894
33.0     134562
50.0     132983
6.0      122587
35.0     121982
47.0     116948
18.0     102789
45.0     102328
37.0      92538
10.0      83283
14.0      75405
2.0       72071
21.0      72037
13.0      70092
39.0      69589
12.0      68709
7.0       62436
32.0      54639
27.0      52783
43.0      51782
17.0      50749
9.0       47746
25.0      46887
16.0      36094
24.0      35426
48.0      35256
26.0      3

#### Variables categóricas

In [47]:
for i in sales_raw.select_dtypes(exclude= ["number"]).columns:
     print(i, sales_raw[i].value_counts())

month_sale 2018-10-01    27693
2018-09-01    24488
2018-08-01    21054
2018-07-01    20602
2018-11-01    19155
2018-02-01    15182
2018-12-01    15015
2019-01-01    11872
2018-03-01    11668
2018-06-01    11558
2019-03-01    11506
2019-02-01    11360
2019-05-01    10211
2018-04-01     9962
2019-04-01     9889
2018-05-01     9558
Name: month_sale, dtype: int64


In [48]:
for i in customer_commercial_activity_raw.select_dtypes(exclude= ["number"]).columns:
  print(i, customer_commercial_activity_raw[i].value_counts())

pk_partition 2019-05    442995
2019-04    439627
2019-03    436183
2019-02    431727
2019-01    426875
2018-12    422481
2018-11    416387
2018-10    402300
2018-09    375323
2018-08    352922
2018-07    339339
2018-06    252104
2018-05    249926
2018-04    247463
2018-03    245258
2018-02    242521
2018-01    239493
Name: pk_partition, dtype: int64
entry_date 2016-10    382473
2017-10    340613
2015-10    310295
2017-09    294752
2017-07    267173
2016-08    252704
2015-08    221984
2015-07    220941
2017-11    217751
2015-09    217174
2018-10    216947
2015-11    210687
2016-09    206107
2018-09    202262
2016-07    189225
2016-11    188601
2018-07    178215
2017-08    162534
2018-08    138420
2017-12    123456
2018-11    103073
2016-12     89423
2015-12     82347
2017-01     65539
2018-02     61283
2018-01     59721
2017-06     54925
2018-03     51022
2016-01     50830
2017-02     50561
2017-03     50268
2017-05     50165
2017-04     47729
2018-04     43480
2018-12     41244
2018-05

In [49]:
for i in customer_sociodemographics_raw.select_dtypes(exclude= ["number"]).columns:
  print(i, customer_sociodemographics_raw[i].value_counts())

pk_partition 2019-05    442995
2019-04    439627
2019-03    436183
2019-02    431727
2019-01    426875
2018-12    422481
2018-11    416387
2018-10    402300
2018-09    375323
2018-08    352922
2018-07    339339
2018-06    252104
2018-05    249926
2018-04    247463
2018-03    245258
2018-02    242521
2018-01    239493
Name: pk_partition, dtype: int64
country_id ES    5960672
GB        441
FR        225
DE        199
US        195
CH        194
BR         87
BE         81
VE         79
IE         68
MX         58
AT         51
AR         51
PL         49
IT         45
MA         34
CL         30
CN         28
CA         22
LU         17
ET         17
QA         17
CI         17
SA         17
CM         17
SN         17
MR         17
NO         17
RU         17
CO         17
GA         17
GT         17
DO         17
SE         16
DJ         11
PT         11
JM         11
RO          9
HU          8
DZ          7
PE          4
Name: country_id, dtype: int64
gender H    3087502
V    2875397

# 3.- PRE PROCESSING

## Procesamiento de Sales

In [50]:
# Merge de Sales y Product Description
#df_sales_raw = sales_raw.merge(product_description_raw, right_on = "pk_product_ID", left_on= "product_ID", how = "left")

In [51]:
df_sales_raw=sales_raw.copy()

In [52]:
sales_raw.shape

(240773, 6)

In [53]:
# Análisis inicial de df_sales_raw y aplicamos las acciones propuestas en el análisis de los datasets

df_sales_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240773 entries, 0 to 240772
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  240773 non-null  int64  
 1   pk_sale     240773 non-null  int64  
 2   cid         240773 non-null  int64  
 3   month_sale  240773 non-null  object 
 4   product_ID  240773 non-null  int64  
 5   net_margin  240773 non-null  float64
dtypes: float64(1), int64(4), object(1)
memory usage: 11.0+ MB


In [54]:
df_sales_raw

,Unnamed: 0,pk_sale,cid,month_sale,product_ID,net_margin
0,0,6666,33620,2018-05-01,2335,952.9
1,1,6667,35063,2018-06-01,2335,1625.2
2,2,6668,37299,2018-02-01,2335,1279.7
3,3,6669,39997,2018-02-01,2335,1511.9
4,4,6670,44012,2018-02-01,2335,1680.3
...,...,...,...,...,...,...
240768,240768,247434,1553456,2019-05-01,4657,56.7
240769,240769,247435,1553541,2019-05-01,4657,66.5
240770,240770,247436,1553559,2019-05-01,4657,73.0
240771,240771,247437,1553565,2019-05-01,4657,82.3


In [55]:
# Quitar pension plan de las variables, solo en target
# Si hay productos/variables muy correlacionadas eliminar también (cerca a 1)

In [56]:
df_sales1 = df_sales_raw.copy()

In [57]:
# Eliminar columna Unnamed

df_sales1.drop("Unnamed: 0", axis = 1, inplace = True)
#df_sales1.drop("Unnamed: 0_y", axis = 1, inplace = True)

# Eliminar columna "pk_product_ID" duplicada tras el merge

#df_sales1.drop("pk_product_ID", axis = 1, inplace = True)

#elimnar todas excepto,  month_sale, customer_id, y product_ID
df_sales1.drop("pk_sale", axis = 1, inplace = True)
df_sales1.drop("net_margin", axis = 1, inplace = True)



In [58]:
df_sales1.head()

,cid,month_sale,product_ID
0,33620,2018-05-01,2335
1,35063,2018-06-01,2335
2,37299,2018-02-01,2335
3,39997,2018-02-01,2335
4,44012,2018-02-01,2335


In [59]:
# Renombrar columnas para alinear el naming, por ejemplo variable cid como "client_ID"

df_sales1.rename({"cid": "customer_ID"},axis='columns',inplace = True)

In [60]:
# Cambiar los identificadores (product_ID, customer_ID) a variable categórica

df_sales1["product_ID"] = df_sales1["product_ID"].astype("object")
df_sales1["customer_ID"] = df_sales1["customer_ID"].astype("object")

In [61]:
# Cambiar tipo de month_sale a fecha

df_sales1["month_sale"] = pd.to_datetime(df_sales1["month_sale"], format = "%Y/%m/%d")

In [62]:
# Disgregar la fecha en mes, año, día, día de la semana (otros: trimestre, fin de semana, festivo...)

#df_sales1["month_sales"] = df_sales1["month_sale"].dt.month
#df_sales1["year_sales"] = df_sales1["month_sale"].dt.year

In [63]:
df_sales1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240773 entries, 0 to 240772
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   customer_ID  240773 non-null  object        
 1   month_sale   240773 non-null  datetime64[ns]
 2   product_ID   240773 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 5.5+ MB


In [64]:
df_sales1['merge_date']=df_sales1['month_sale']-pd.DateOffset(months=1)


In [65]:
df_sales1.head()

,customer_ID,month_sale,product_ID,merge_date
0,33620,2018-05-01,2335,2018-04-01
1,35063,2018-06-01,2335,2018-05-01
2,37299,2018-02-01,2335,2018-01-01
3,39997,2018-02-01,2335,2018-01-01
4,44012,2018-02-01,2335,2018-01-01


In [66]:
df_sales1.shape

(240773, 4)

## Procesamiento de Customer Sociodemographics

In [67]:
customer_sociodemographics_raw.head()

,Unnamed: 0,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary
0,0,1375586,2018-01,ES,29.0,H,35,N,87218.10
1,1,1050611,2018-01,ES,13.0,V,23,N,35548.74
2,2,1050612,2018-01,ES,13.0,V,23,N,122179.11
3,3,1050613,2018-01,ES,50.0,H,22,N,119775.54
4,4,1050614,2018-01,ES,50.0,V,23,N,NaN


In [68]:
#hacemos una copia del dataset par eliminar los deceased yes
df_soc=customer_sociodemographics_raw[customer_sociodemographics_raw['deceased']=='N'].copy()

In [69]:
df_soc.drop("Unnamed: 0", axis = 1, inplace = True)
df_soc.drop("deceased", axis = 1, inplace = True)

In [70]:
df_soc.rename({"pk_cid": "customer_ID",'pk_partition':'partition_ID'},axis='columns',inplace = True)

In [71]:
df_soc.head()

,customer_ID,partition_ID,country_id,region_code,gender,age,salary
0,1375586,2018-01,ES,29.0,H,35,87218.10
1,1050611,2018-01,ES,13.0,V,23,35548.74
2,1050612,2018-01,ES,13.0,V,23,122179.11
3,1050613,2018-01,ES,50.0,H,22,119775.54
4,1050614,2018-01,ES,50.0,V,23,NaN


In [72]:
df_soc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5961849 entries, 0 to 5962923
Data columns (total 7 columns):
 #   Column        Dtype  
---  ------        -----  
 0   customer_ID   int64  
 1   partition_ID  object 
 2   country_id    object 
 3   region_code   float64
 4   gender        object 
 5   age           int64  
 6   salary        float64
dtypes: float64(2), int64(2), object(3)
memory usage: 363.9+ MB


In [73]:
df_soc.shape

(5961849, 7)

## Customer commercial activity

In [74]:
customer_commercial_activity_raw.head()

,Unnamed: 0,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment
0,0,1375586,2018-01,2018-01,KHL,1.0,02 - PARTICULARES
1,1,1050611,2018-01,2015-08,KHE,0.0,03 - UNIVERSITARIO
2,2,1050612,2018-01,2015-08,KHE,0.0,03 - UNIVERSITARIO
3,3,1050613,2018-01,2015-08,KHD,0.0,03 - UNIVERSITARIO
4,4,1050614,2018-01,2015-08,KHE,1.0,03 - UNIVERSITARIO


In [75]:
cust_com=customer_commercial_activity_raw.copy()

In [76]:
cust_com.drop("Unnamed: 0", axis = 1, inplace = True)


In [77]:
#sustituimos entry_date por antiguedad

In [78]:
cust_com['pk_partition'].max()

'2019-05'

In [79]:
cust_com['antiguedad']=pd.to_datetime(cust_com['pk_partition'].max())-pd.to_datetime(cust_com['entry_date'])

In [80]:
cust_com.head()

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,antiguedad
0,1375586,2018-01,2018-01,KHL,1.0,02 - PARTICULARES,485 days
1,1050611,2018-01,2015-08,KHE,0.0,03 - UNIVERSITARIO,1369 days
2,1050612,2018-01,2015-08,KHE,0.0,03 - UNIVERSITARIO,1369 days
3,1050613,2018-01,2015-08,KHD,0.0,03 - UNIVERSITARIO,1369 days
4,1050614,2018-01,2015-08,KHE,1.0,03 - UNIVERSITARIO,1369 days


In [81]:
cust_com.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 7 columns):
 #   Column           Dtype          
---  ------           -----          
 0   pk_cid           int64          
 1   pk_partition     object         
 2   entry_date       object         
 3   entry_channel    object         
 4   active_customer  float64        
 5   segment          object         
 6   antiguedad       timedelta64[ns]
dtypes: float64(1), int64(1), object(4), timedelta64[ns](1)
memory usage: 318.5+ MB


In [82]:
cust_com['antiguedad'].astype(int)

0           41904000000000000
1          118281600000000000
2          118281600000000000
3          118281600000000000
4          118281600000000000
                  ...        
5962919     86659200000000000
5962920     86659200000000000
5962921     86659200000000000
5962922     86659200000000000
5962923                     0
Name: antiguedad, Length: 5962924, dtype: int64

In [83]:
cust_com['antiguedad']=cust_com['antiguedad'].astype(int)

In [84]:
cust_com['antiguedad']/86400000000000

0           485.0
1          1369.0
2          1369.0
3          1369.0
4          1369.0
            ...  
5962919    1003.0
5962920    1003.0
5962921    1003.0
5962922    1003.0
5962923       0.0
Name: antiguedad, Length: 5962924, dtype: float64

In [85]:
cust_com['antiguedad']=cust_com['antiguedad']/86400000000000

In [86]:
cust_com.head()

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,antiguedad
0,1375586,2018-01,2018-01,KHL,1.0,02 - PARTICULARES,485.0
1,1050611,2018-01,2015-08,KHE,0.0,03 - UNIVERSITARIO,1369.0
2,1050612,2018-01,2015-08,KHE,0.0,03 - UNIVERSITARIO,1369.0
3,1050613,2018-01,2015-08,KHD,0.0,03 - UNIVERSITARIO,1369.0
4,1050614,2018-01,2015-08,KHE,1.0,03 - UNIVERSITARIO,1369.0


In [87]:
cust_com.rename({"pk_cid": "customer_ID",'pk_partition':'partition_ID'},axis='columns',inplace = True)

In [88]:
customer_comercial_activity=cust_com[cust_com['active_customer']==1.0].copy()

In [89]:
customer_comercial_activity.drop("active_customer", axis = 1, inplace = True)
customer_comercial_activity.drop("entry_date", axis = 1, inplace = True)

In [90]:
customer_comercial_activity.head()

,customer_ID,partition_ID,entry_channel,segment,antiguedad
0,1375586,2018-01,KHL,02 - PARTICULARES,485.0
4,1050614,2018-01,KHE,03 - UNIVERSITARIO,1369.0
7,1050617,2018-01,KHE,03 - UNIVERSITARIO,1369.0
12,1050623,2018-01,KHE,03 - UNIVERSITARIO,1369.0
13,1050624,2018-01,KHE,02 - PARTICULARES,1369.0


In [91]:
customer_comercial_activity.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2405351 entries, 0 to 5962923
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   customer_ID    int64  
 1   partition_ID   object 
 2   entry_channel  object 
 3   segment        object 
 4   antiguedad     float64
dtypes: float64(1), int64(1), object(3)
memory usage: 110.1+ MB


In [92]:
customer_comercial_activity.shape

(2405351, 5)

## Customer products

In [93]:
customer_products_raw.head()

,Unnamed: 0,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,0,1375586,2018-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
1,1,1050611,2018-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
2,2,1050612,2018-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
3,3,1050613,2018-01,1,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0
4,4,1050614,2018-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1


In [94]:
customer_products_raw.shape

(5962924, 18)

In [95]:
df_customer_products=customer_products_raw.copy()

In [96]:
df_customer_products.drop("Unnamed: 0", axis = 1, inplace = True)
df_customer_products.drop("pension_plan", axis = 1, inplace = True)

In [97]:
df_customer_products.rename({"pk_cid": "customer_ID",'pk_partition':'partition_ID'},axis='columns',inplace = True)

In [98]:
df_customer_products.shape

(5962924, 16)

## Merge tablas customer sociodemographics y customer commercial activity

In [99]:
df_soc.head()

,customer_ID,partition_ID,country_id,region_code,gender,age,salary
0,1375586,2018-01,ES,29.0,H,35,87218.10
1,1050611,2018-01,ES,13.0,V,23,35548.74
2,1050612,2018-01,ES,13.0,V,23,122179.11
3,1050613,2018-01,ES,50.0,H,22,119775.54
4,1050614,2018-01,ES,50.0,V,23,NaN


In [100]:
customer_comercial_activity.head()

,customer_ID,partition_ID,entry_channel,segment,antiguedad
0,1375586,2018-01,KHL,02 - PARTICULARES,485.0
4,1050614,2018-01,KHE,03 - UNIVERSITARIO,1369.0
7,1050617,2018-01,KHE,03 - UNIVERSITARIO,1369.0
12,1050623,2018-01,KHE,03 - UNIVERSITARIO,1369.0
13,1050624,2018-01,KHE,02 - PARTICULARES,1369.0


In [144]:

customer_comercial_activity=customer_comercial_activity.reset_index()

In [145]:
customer_comercial_activity.head()

,index,customer_ID,partition_ID,entry_channel,segment,antiguedad
0,0,1375586,2018-01,KHL,02 - PARTICULARES,485.0
1,4,1050614,2018-01,KHE,03 - UNIVERSITARIO,1369.0
2,7,1050617,2018-01,KHE,03 - UNIVERSITARIO,1369.0
3,12,1050623,2018-01,KHE,03 - UNIVERSITARIO,1369.0
4,13,1050624,2018-01,KHE,02 - PARTICULARES,1369.0


In [146]:
del customer_comercial_activity['index']

In [147]:
customer_comercial_activity.head()

,customer_ID,partition_ID,entry_channel,segment,antiguedad
0,1375586,2018-01,KHL,02 - PARTICULARES,485.0
1,1050614,2018-01,KHE,03 - UNIVERSITARIO,1369.0
2,1050617,2018-01,KHE,03 - UNIVERSITARIO,1369.0
3,1050623,2018-01,KHE,03 - UNIVERSITARIO,1369.0
4,1050624,2018-01,KHE,02 - PARTICULARES,1369.0


In [106]:
customer_comercial_activity.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2405351 entries, 0 to 5962923
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   customer_ID    int64  
 1   partition_ID   object 
 2   entry_channel  object 
 3   segment        object 
 4   antiguedad     float64
dtypes: float64(1), int64(1), object(3)
memory usage: 110.1+ MB


In [96]:
# Merge de las tablas relacionadas con el cliente
#df_raw = customer_commercial_activity_raw.merge(customer_sociodemographics_raw, on=["pk_cid","pk_partition"])\
            #.merge(customer_products_raw, on=["pk_cid","pk_partition"])

In [109]:
#Merge de customer_commercial_activity y customer_sociodemografics, customer_products lo dejamos para más tarde
#hago un left join con la tabla customer_commercial activity primero ya que tiene menos filas y son los clientes activos

df1_raw=customer_comercial_activity.merge(df_soc, on=["customer_ID","partition_ID"])

In [107]:
customer_comercial_activity.shape

(2405351, 5)

In [108]:
df_soc.shape

(5961849, 7)

In [110]:
df1_raw.shape

(2404633, 10)

In [111]:
df1_raw.head()

,customer_ID,partition_ID,entry_channel,segment,antiguedad,country_id,region_code,gender,age,salary
0,1375586,2018-01,KHL,02 - PARTICULARES,485.0,ES,29.0,H,35,87218.10
1,1050614,2018-01,KHE,03 - UNIVERSITARIO,1369.0,ES,50.0,V,23,NaN
2,1050617,2018-01,KHE,03 - UNIVERSITARIO,1369.0,ES,50.0,H,23,113316.66
3,1050623,2018-01,KHE,03 - UNIVERSITARIO,1369.0,ES,49.0,H,23,113538.81
4,1050624,2018-01,KHE,02 - PARTICULARES,1369.0,ES,50.0,H,65,61605.09


In [112]:
df1_raw.isnull().sum()

customer_ID           0
partition_ID          0
entry_channel     42809
segment           43176
antiguedad            0
country_id            0
region_code        1501
gender               25
age                   0
salary           596610
dtype: int64

In [113]:
# Análisis inicial de df_sales_raw y aplicamos las acciones propuestas en el análisis de los datasets

df1_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2404633 entries, 0 to 2404632
Data columns (total 10 columns):
 #   Column         Dtype  
---  ------         -----  
 0   customer_ID    int64  
 1   partition_ID   object 
 2   entry_channel  object 
 3   segment        object 
 4   antiguedad     float64
 5   country_id     object 
 6   region_code    float64
 7   gender         object 
 8   age            int64  
 9   salary         float64
dtypes: float64(3), int64(2), object(5)
memory usage: 201.8+ MB


In [114]:
df1 = df1_raw.copy()

In [115]:
df1.shape

(2404633, 10)

In [116]:
df1.head()

,customer_ID,partition_ID,entry_channel,segment,antiguedad,country_id,region_code,gender,age,salary
0,1375586,2018-01,KHL,02 - PARTICULARES,485.0,ES,29.0,H,35,87218.10
1,1050614,2018-01,KHE,03 - UNIVERSITARIO,1369.0,ES,50.0,V,23,NaN
2,1050617,2018-01,KHE,03 - UNIVERSITARIO,1369.0,ES,50.0,H,23,113316.66
3,1050623,2018-01,KHE,03 - UNIVERSITARIO,1369.0,ES,49.0,H,23,113538.81
4,1050624,2018-01,KHE,02 - PARTICULARES,1369.0,ES,50.0,H,65,61605.09


### Atributos de baja varianza

In [117]:
df1["country_id"].value_counts(normalize=True)*100

ES    99.938078
GB     0.014930
DE     0.007818
FR     0.006654
CH     0.005697
US     0.004200
IE     0.002121
AT     0.002121
VE     0.001705
AR     0.001663
MX     0.001497
CL     0.001248
CN     0.001123
PL     0.000873
BR     0.000832
BE     0.000832
IT     0.000790
SA     0.000707
LU     0.000707
QA     0.000707
CI     0.000707
GT     0.000707
CM     0.000707
ET     0.000707
NO     0.000707
DO     0.000707
CA     0.000665
RO     0.000374
DZ     0.000250
PE     0.000166
Name: country_id, dtype: float64

In [118]:
#agrupamos country_id en ES y OTHERS
df1['country_id'] = df1['country_id'].apply(lambda x: 'ES' if x == 'ES' else 'OTHERS')

In [119]:
df1["country_id"].value_counts(normalize=True)*100

ES        99.938078
OTHERS     0.061922
Name: country_id, dtype: float64

In [110]:
#df1.drop("country_id", axis = 1, inplace = True)

### Gestión de nulos

In [120]:
df1.isnull().sum()

customer_ID           0
partition_ID          0
entry_channel     42809
segment           43176
antiguedad            0
country_id            0
region_code        1501
gender               25
age                   0
salary           596610
dtype: int64

In [121]:
df1['salary'].sort_values(ascending = False)

1183631    28894395.51
873103     28894395.51
1642390    28894395.51
2146645    28894395.51
800737     28894395.51
              ...     
2404620            NaN
2404627            NaN
2404629            NaN
2404631            NaN
2404632            NaN
Name: salary, Length: 2404633, dtype: float64

In [123]:
# Asignamos valor extremos a "salary", "gender", "region_code" ya que vamos a hacer un DecisionTree

print(df1["salary"].describe())
df1["salary"].fillna(value = 0, inplace = True)

print(df1["gender"].value_counts(normalize = True))
df1["gender"].fillna(value = 2, inplace = True)

print(df1["region_code"].describe())
df1["region_code"].fillna(value = 1000, inplace = True)

count    2.404633e+06
mean     8.794045e+04
std      1.861135e+05
min      0.000000e+00
25%      1.754619e+04
50%      7.254840e+04
75%      1.162219e+05
max      2.889440e+07
Name: salary, dtype: float64
V    0.51776
H    0.48223
2    0.00001
Name: gender, dtype: float64
count    2.404633e+06
mean     2.688807e+01
std      2.776481e+01
min      1.000000e+00
25%      1.500000e+01
50%      2.800000e+01
75%      3.600000e+01
max      1.000000e+03
Name: region_code, dtype: float64


#### Segment (sustituir por valores exisitnetes en otras particiones)

In [124]:
# Array con los ids que tienen nulos en "segment"
ids_nullsegment_unique=df1[(df1['segment'].isnull()==True)]['customer_ID'].unique()

In [125]:
# Dataframe donde revisamos si en la última carga de datos hay un valor para los ids que tienen "segment" nulo. 
valores_nulos_segment=df1[(df1.customer_ID.isin(ids_nullsegment_unique))&(df1['segment'].notnull())&(df1['partition_ID']== "2019-05")][['customer_ID','segment']]

In [126]:
# Si el id con "segment" nulo tenía un valor correcto en la última partition, se lo asignaremos a los valores nulos.
dict_nulos_segment=dict(zip(valores_nulos_segment['customer_ID'],valores_nulos_segment['segment']))

#### Entry_Chanel (sustituir por valores exisitnetes en otras particiones)

In [127]:
# Array con los ids que tienen nulos en "entry_channel"
ids_nullentrychannel_unique=df1[(df1['entry_channel'].isnull()==True)]['customer_ID'].unique()

In [128]:
len(ids_nullentrychannel_unique)

42746

In [129]:
# Dataframe donde revisamos si en la última carga de datos hay un valor para los ids que tienen "entry_channel" nulo. 
valores_nulos_entrychannel=df1[(df1.customer_ID.isin(ids_nullentrychannel_unique))&(df1['entry_channel'].notnull())&(df1['partition_ID']=='2019-05')][['customer_ID','entry_channel']]

In [130]:
# Si el id con "entry_channel" nulo tenía un valor correcto en la última partition, se lo asignaremos a los valores nulos.
dict_nulos_entrychannel=dict(zip(valores_nulos_entrychannel['customer_ID'],valores_nulos_entrychannel['entry_channel']))

In [131]:
df1.set_index('customer_ID',inplace = True)

In [132]:
# Imputamos los últimos valores registrados correctamente en "segment" y "entry_channel"
df1['segment'].fillna(dict_nulos_segment,inplace=True)
df1['entry_channel'].fillna(dict_nulos_entrychannel,inplace=True)

In [133]:
df1.isnull().sum()

partition_ID        0
entry_channel    8136
segment          8201
antiguedad          0
country_id          0
region_code         0
gender              0
age                 0
salary              0
dtype: int64

In [134]:
# Imputación de valores extremos a los nulos que no se han podido deducir por el histórico
df1['segment'].fillna(value='04 - NO SEGMENT',inplace=True)
df1['entry_channel'].fillna(value='ZZZ',inplace=True)

In [135]:
df1['entry_channel'].value_counts()

KHE    788510
KFC    569947
KAT    277627
KHQ    197304
KHK    147787
KHM    129104
KHN     87927
KFA     68004
RED     53686
KHL     29443
KHD     22234
ZZZ      8136
KHO      7025
KHF      6080
KHC      3920
KAZ      3729
KBG      1253
007       568
013       318
KAG       250
KHP       223
KAS       216
KBZ       168
KEH       135
KAA       128
KAY       100
KAR        82
KAF        78
KFD        60
KCC        50
KAD        45
KAB        41
KCB        39
KAE        39
KAW        36
KAK        34
KAM        30
KAQ        26
KBO        17
KGN        17
KBH        17
KFS        17
KAH        17
004        17
KBE        17
KEY        16
KCL        16
KDT        15
KFL        13
KBW        12
KAJ        12
KES        11
KDR        10
KCK        10
KCH         8
KCI         7
KDA         2
Name: entry_channel, dtype: int64

In [136]:
def setOthers(dataframe, column, num_values=5):
    top_categories = dataframe[column].value_counts().head(num_values)
    top_categories_list = top_categories.index.to_list()
    top_categories_list.append('OTHERS')
    dataframe[column] = pd.Categorical(dataframe[column], categories=top_categories_list)
    return dataframe[column].fillna('OTHERS')

In [137]:
df1['entry_channel']=setOthers(df1,'entry_channel',num_values=13)

In [138]:
df1['entry_channel'].value_counts()

KHE       788510
KFC       569947
KAT       277627
KHQ       197304
KHK       147787
KHM       129104
KHN        87927
KFA        68004
RED        53686
KHL        29443
KHD        22234
OTHERS     17899
ZZZ         8136
KHO         7025
Name: entry_channel, dtype: int64

In [139]:
df1.isnull().sum()

partition_ID     0
entry_channel    0
segment          0
antiguedad       0
country_id       0
region_code      0
gender           0
age              0
salary           0
dtype: int64

In [140]:
df1.head()

,partition_ID,entry_channel,segment,antiguedad,country_id,region_code,gender,age,salary
customer_ID,,,,,,,,,
1375586,2018-01,KHL,02 - PARTICULARES,485.0,ES,29.0,H,35,87218.10
1050614,2018-01,KHE,03 - UNIVERSITARIO,1369.0,ES,50.0,V,23,0.00
1050617,2018-01,KHE,03 - UNIVERSITARIO,1369.0,ES,50.0,H,23,113316.66
1050623,2018-01,KHE,03 - UNIVERSITARIO,1369.0,ES,49.0,H,23,113538.81
1050624,2018-01,KHE,02 - PARTICULARES,1369.0,ES,50.0,H,65,61605.09


In [141]:
df1=df1.reset_index()

In [142]:
df1.head()

,customer_ID,partition_ID,entry_channel,segment,antiguedad,country_id,region_code,gender,age,salary
0,1375586,2018-01,KHL,02 - PARTICULARES,485.0,ES,29.0,H,35,87218.10
1,1050614,2018-01,KHE,03 - UNIVERSITARIO,1369.0,ES,50.0,V,23,0.00
2,1050617,2018-01,KHE,03 - UNIVERSITARIO,1369.0,ES,50.0,H,23,113316.66
3,1050623,2018-01,KHE,03 - UNIVERSITARIO,1369.0,ES,49.0,H,23,113538.81
4,1050624,2018-01,KHE,02 - PARTICULARES,1369.0,ES,50.0,H,65,61605.09


In [143]:
df1.shape

(2404633, 10)

## Preprocesamiento de Customer Products

In [148]:
df_customer_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 16 columns):
 #   Column              Dtype  
---  ------              -----  
 0   customer_ID         int64  
 1   partition_ID        object 
 2   short_term_deposit  int64  
 3   loans               int64  
 4   mortgage            int64  
 5   funds               int64  
 6   securities          int64  
 7   long_term_deposit   int64  
 8   em_account_pp       int64  
 9   credit_card         int64  
 10  payroll             float64
 11  payroll_account     int64  
 12  emc_account         int64  
 13  debit_card          int64  
 14  em_account_p        int64  
 15  em_acount           int64  
dtypes: float64(1), int64(14), object(1)
memory usage: 727.9+ MB


In [149]:
df_customer_products.head()

,customer_ID,partition_ID,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,1375586,2018-01,0,0,0,0,0,0,0,0,0.0,0,0,0,0,1
1,1050611,2018-01,0,0,0,0,0,0,0,0,0.0,0,0,0,0,1
2,1050612,2018-01,0,0,0,0,0,0,0,0,0.0,0,0,0,0,1
3,1050613,2018-01,1,0,0,0,0,0,0,0,0.0,0,0,0,0,0
4,1050614,2018-01,0,0,0,0,0,0,0,0,0.0,0,0,0,0,1


In [150]:
df_customer_products["em_account_p"].value_counts(normalize = True)*100

0    99.99943
1     0.00057
Name: em_account_p, dtype: float64

In [151]:
df_customer_products["em_account_pp"].value_counts(normalize = True)*100

0    100.0
Name: em_account_pp, dtype: float64

In [152]:
# Eliminamos variables de baja varianza
del df_customer_products['em_account_pp']
del df_customer_products['em_account_p']


In [153]:
# Eliminamos valores nulos de "payroll" y "pension_plan"

df_customer_products.dropna(subset = ['payroll'], inplace = True)

In [154]:
df_customer_products.isnull().sum()

customer_ID           0
partition_ID          0
short_term_deposit    0
loans                 0
mortgage              0
funds                 0
securities            0
long_term_deposit     0
credit_card           0
payroll               0
payroll_account       0
emc_account           0
debit_card            0
em_acount             0
dtype: int64

In [155]:
df_customer_products.shape

(5962863, 14)

In [156]:
df_customer_products.head()

,customer_ID,partition_ID,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,credit_card,payroll,payroll_account,emc_account,debit_card,em_acount
0,1375586,2018-01,0,0,0,0,0,0,0,0.0,0,0,0,1
1,1050611,2018-01,0,0,0,0,0,0,0,0.0,0,0,0,1
2,1050612,2018-01,0,0,0,0,0,0,0,0.0,0,0,0,1
3,1050613,2018-01,1,0,0,0,0,0,0,0.0,0,0,0,0
4,1050614,2018-01,0,0,0,0,0,0,0,0.0,0,0,0,1


In [157]:
df_customer_products.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5962863 entries, 0 to 5962923
Columns: 14 entries, customer_ID to em_acount
dtypes: float64(1), int64(12), object(1)
memory usage: 682.4+ MB


### Transformación de fechas

In [158]:
df1.head()

,customer_ID,partition_ID,entry_channel,segment,antiguedad,country_id,region_code,gender,age,salary
0,1375586,2018-01,KHL,02 - PARTICULARES,485.0,ES,29.0,H,35,87218.10
1,1050614,2018-01,KHE,03 - UNIVERSITARIO,1369.0,ES,50.0,V,23,0.00
2,1050617,2018-01,KHE,03 - UNIVERSITARIO,1369.0,ES,50.0,H,23,113316.66
3,1050623,2018-01,KHE,03 - UNIVERSITARIO,1369.0,ES,49.0,H,23,113538.81
4,1050624,2018-01,KHE,02 - PARTICULARES,1369.0,ES,50.0,H,65,61605.09


In [159]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2404633 entries, 0 to 2404632
Data columns (total 10 columns):
 #   Column         Dtype   
---  ------         -----   
 0   customer_ID    int64   
 1   partition_ID   object  
 2   entry_channel  category
 3   segment        object  
 4   antiguedad     float64 
 5   country_id     object  
 6   region_code    float64 
 7   gender         object  
 8   age            int64   
 9   salary         float64 
dtypes: category(1), float64(3), int64(2), object(4)
memory usage: 167.4+ MB


In [160]:
df_customer_products.head()

,customer_ID,partition_ID,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,credit_card,payroll,payroll_account,emc_account,debit_card,em_acount
0,1375586,2018-01,0,0,0,0,0,0,0,0.0,0,0,0,1
1,1050611,2018-01,0,0,0,0,0,0,0,0.0,0,0,0,1
2,1050612,2018-01,0,0,0,0,0,0,0,0.0,0,0,0,1
3,1050613,2018-01,1,0,0,0,0,0,0,0.0,0,0,0,0
4,1050614,2018-01,0,0,0,0,0,0,0,0.0,0,0,0,1


In [161]:
df_customer_products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5962863 entries, 0 to 5962923
Data columns (total 14 columns):
 #   Column              Dtype  
---  ------              -----  
 0   customer_ID         int64  
 1   partition_ID        object 
 2   short_term_deposit  int64  
 3   loans               int64  
 4   mortgage            int64  
 5   funds               int64  
 6   securities          int64  
 7   long_term_deposit   int64  
 8   credit_card         int64  
 9   payroll             float64
 10  payroll_account     int64  
 11  emc_account         int64  
 12  debit_card          int64  
 13  em_acount           int64  
dtypes: float64(1), int64(12), object(1)
memory usage: 682.4+ MB


In [162]:

df1['partition_ID']=pd.to_datetime(df1['partition_ID'])
df_customer_products['partition_ID'] = pd.to_datetime(df_customer_products['partition_ID'])



In [163]:
df_customer_products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5962863 entries, 0 to 5962923
Data columns (total 14 columns):
 #   Column              Dtype         
---  ------              -----         
 0   customer_ID         int64         
 1   partition_ID        datetime64[ns]
 2   short_term_deposit  int64         
 3   loans               int64         
 4   mortgage            int64         
 5   funds               int64         
 6   securities          int64         
 7   long_term_deposit   int64         
 8   credit_card         int64         
 9   payroll             float64       
 10  payroll_account     int64         
 11  emc_account         int64         
 12  debit_card          int64         
 13  em_acount           int64         
dtypes: datetime64[ns](1), float64(1), int64(12)
memory usage: 682.4 MB


In [164]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2404633 entries, 0 to 2404632
Data columns (total 10 columns):
 #   Column         Dtype         
---  ------         -----         
 0   customer_ID    int64         
 1   partition_ID   datetime64[ns]
 2   entry_channel  category      
 3   segment        object        
 4   antiguedad     float64       
 5   country_id     object        
 6   region_code    float64       
 7   gender         object        
 8   age            int64         
 9   salary         float64       
dtypes: category(1), datetime64[ns](1), float64(3), int64(2), object(3)
memory usage: 167.4+ MB


### Tranformación de numéricas

In [165]:
# Cambiamos el type de las variables con float a int (active_customer, payroll y pension_plan), excepto salary, que si que es float

variables_to_int_customer_products = [ 'payroll']
#variables_to_int_df1 = ['active_customer']

df_customer_products[variables_to_int_customer_products] = df_customer_products[variables_to_int_customer_products].applymap(int)
#df1[variables_to_int_df1] = df1[variables_to_int_df1].applymap(int)

In [166]:
df_sales2=df_sales1.to_pickle('/content/drive/MyDrive/TFM  Data Science/df_sales2')


In [167]:
df_sales2=pd.read_pickle('/content/drive/MyDrive/TFM  Data Science/df_sales2') #Drive de Eva
#df_sales2=pd.read_pickle('/content/drive/MyDrive/Master_DS/TFM_Data_Science/df_sales2') #Drive de Culebra

## Merge tabla sales y resto del dataset

In [ ]:
#hago merge de df_sales 2 que tiene 240773 filas ( son todas las compras) con la tabla df1 que es la información de clientes a fecha de venta con la partitio_ID de la table df1.
#la fecha merge_date la utilizaré para mergear con la tabla de customer_products proque quiero la información de un mes antes para que el modelo prediga mejor

In [174]:
df_sales2.head()

,customer_ID,month_sale,product_ID,merge_date
0,33620,2018-05-01,2335,2018-04-01
1,35063,2018-06-01,2335,2018-05-01
2,37299,2018-02-01,2335,2018-01-01
3,39997,2018-02-01,2335,2018-01-01
4,44012,2018-02-01,2335,2018-01-01


In [175]:
df_sales2.shape

(240773, 4)

In [176]:
df_sales2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240773 entries, 0 to 240772
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   customer_ID  240773 non-null  int64         
 1   month_sale   240773 non-null  datetime64[ns]
 2   product_ID   240773 non-null  object        
 3   merge_date   240773 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(1)
memory usage: 7.3+ MB


In [177]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2404633 entries, 0 to 2404632
Data columns (total 10 columns):
 #   Column         Dtype         
---  ------         -----         
 0   customer_ID    int64         
 1   partition_ID   datetime64[ns]
 2   entry_channel  category      
 3   segment        object        
 4   antiguedad     float64       
 5   country_id     object        
 6   region_code    float64       
 7   gender         object        
 8   age            int64         
 9   salary         float64       
dtypes: category(1), datetime64[ns](1), float64(3), int64(2), object(3)
memory usage: 167.4+ MB


In [178]:
df_sales2['customer_ID']=df_sales2['customer_ID'].astype('int')

In [172]:
df1.head()

,customer_ID,partition_ID,entry_channel,segment,antiguedad,country_id,region_code,gender,age,salary
0,1375586,2018-01-01,KHL,02 - PARTICULARES,485.0,ES,29.0,H,35,87218.10
1,1050614,2018-01-01,KHE,03 - UNIVERSITARIO,1369.0,ES,50.0,V,23,0.00
2,1050617,2018-01-01,KHE,03 - UNIVERSITARIO,1369.0,ES,50.0,H,23,113316.66
3,1050623,2018-01-01,KHE,03 - UNIVERSITARIO,1369.0,ES,49.0,H,23,113538.81
4,1050624,2018-01-01,KHE,02 - PARTICULARES,1369.0,ES,50.0,H,65,61605.09


In [180]:
df_total1 = df_sales2.merge(df1,left_on= ['customer_ID',"month_sale"], right_on = ['customer_ID', 'partition_ID'], how = "left")


In [181]:
df_total1.shape

(240773, 13)

In [182]:
df_total1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240773 entries, 0 to 240772
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   customer_ID    240773 non-null  int64         
 1   month_sale     240773 non-null  datetime64[ns]
 2   product_ID     240773 non-null  object        
 3   merge_date     240773 non-null  datetime64[ns]
 4   partition_ID   191856 non-null  datetime64[ns]
 5   entry_channel  191856 non-null  category      
 6   segment        191856 non-null  object        
 7   antiguedad     191856 non-null  float64       
 8   country_id     191856 non-null  object        
 9   region_code    191856 non-null  float64       
 10  gender         191856 non-null  object        
 11  age            191856 non-null  float64       
 12  salary         191856 non-null  float64       
dtypes: category(1), datetime64[ns](3), float64(4), int64(1), object(4)
memory usage: 24.1+ MB


In [183]:
df_total1.isnull().sum()

customer_ID          0
month_sale           0
product_ID           0
merge_date           0
partition_ID     48917
entry_channel    48917
segment          48917
antiguedad       48917
country_id       48917
region_code      48917
gender           48917
age              48917
salary           48917
dtype: int64

In [184]:
df_total1.head()

,customer_ID,month_sale,product_ID,merge_date,partition_ID,entry_channel,segment,antiguedad,country_id,region_code,gender,age,salary
0,33620,2018-05-01,2335,2018-04-01,2018-05-01,KHK,02 - PARTICULARES,395.0,ES,28.0,H,50.0,138618.57
1,35063,2018-06-01,2335,2018-05-01,2018-06-01,KHN,01 - TOP,395.0,ES,28.0,H,62.0,125530.41
2,37299,2018-02-01,2335,2018-01-01,2018-02-01,KHK,01 - TOP,485.0,ES,30.0,V,54.0,0.00
3,39997,2018-02-01,2335,2018-01-01,2018-02-01,KAT,02 - PARTICULARES,485.0,ES,41.0,V,62.0,201575.01
4,44012,2018-02-01,2335,2018-01-01,2018-02-01,KHL,02 - PARTICULARES,485.0,ES,28.0,V,42.0,97601.04


In [185]:
df_total1[df_total1['partition_ID'].isnull()]

,customer_ID,month_sale,product_ID,merge_date,partition_ID,entry_channel,segment,antiguedad,country_id,region_code,gender,age,salary
13,168399,2018-03-01,2335,2018-02-01,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,205204,2018-04-01,2335,2018-03-01,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,328494,2018-04-01,2335,2018-03-01,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,336613,2018-03-01,2335,2018-02-01,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,525304,2018-03-01,2335,2018-02-01,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
240744,1553119,2019-05-01,4657,2019-04-01,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
240747,1553152,2019-05-01,4657,2019-04-01,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
240750,1553188,2019-05-01,4657,2019-04-01,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
240761,1553338,2019-05-01,4657,2019-04-01,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#los nulos deben ser clientes que o están muertos o no están activos, con lo cual los eliminamos

In [186]:
df_total1.dropna( inplace = True)

In [187]:
df_total1.isnull().sum()

customer_ID      0
month_sale       0
product_ID       0
merge_date       0
partition_ID     0
entry_channel    0
segment          0
antiguedad       0
country_id       0
region_code      0
gender           0
age              0
salary           0
dtype: int64

In [188]:
df_total1.shape

(191856, 13)

In [189]:
df_customer_products.head()

,customer_ID,partition_ID,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,credit_card,payroll,payroll_account,emc_account,debit_card,em_acount
0,1375586,2018-01-01,0,0,0,0,0,0,0,0,0,0,0,1
1,1050611,2018-01-01,0,0,0,0,0,0,0,0,0,0,0,1
2,1050612,2018-01-01,0,0,0,0,0,0,0,0,0,0,0,1
3,1050613,2018-01-01,1,0,0,0,0,0,0,0,0,0,0,0
4,1050614,2018-01-01,0,0,0,0,0,0,0,0,0,0,0,1


In [192]:
df_total1.head()

,customer_ID,month_sale,product_ID,merge_date,partition_ID,entry_channel,segment,antiguedad,country_id,region_code,gender,age,salary
0,33620,2018-05-01,2335,2018-04-01,2018-05-01,KHK,02 - PARTICULARES,395.0,ES,28.0,H,50.0,138618.57
1,35063,2018-06-01,2335,2018-05-01,2018-06-01,KHN,01 - TOP,395.0,ES,28.0,H,62.0,125530.41
2,37299,2018-02-01,2335,2018-01-01,2018-02-01,KHK,01 - TOP,485.0,ES,30.0,V,54.0,0.00
3,39997,2018-02-01,2335,2018-01-01,2018-02-01,KAT,02 - PARTICULARES,485.0,ES,41.0,V,62.0,201575.01
4,44012,2018-02-01,2335,2018-01-01,2018-02-01,KHL,02 - PARTICULARES,485.0,ES,28.0,V,42.0,97601.04


In [190]:
df_customer_products.shape

(5962863, 14)

In [191]:
df_total1.shape

(191856, 13)

In [193]:
#hacemos merge de la tabla sales y la tabla customer products con merge_date y partition_id

df_total2=df_total1.merge(df_customer_products,  left_on= ['customer_ID',"merge_date"],right_on = ['customer_ID', 'partition_ID'], how = "left")


In [194]:
df_total2.shape

(191856, 26)

In [195]:
df_total2.head()

,customer_ID,month_sale,product_ID,merge_date,partition_ID_x,entry_channel,segment,antiguedad,country_id,region_code,...,mortgage,funds,securities,long_term_deposit,credit_card,payroll,payroll_account,emc_account,debit_card,em_acount
0,33620,2018-05-01,2335,2018-04-01,2018-05-01,KHK,02 - PARTICULARES,395.0,ES,28.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,35063,2018-06-01,2335,2018-05-01,2018-06-01,KHN,01 - TOP,395.0,ES,28.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,37299,2018-02-01,2335,2018-01-01,2018-02-01,KHK,01 - TOP,485.0,ES,30.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,39997,2018-02-01,2335,2018-01-01,2018-02-01,KAT,02 - PARTICULARES,485.0,ES,41.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,44012,2018-02-01,2335,2018-01-01,2018-02-01,KHL,02 - PARTICULARES,485.0,ES,28.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [196]:
df_total2.head().T

,0,1,2,3,4
customer_ID,33620,35063,37299,39997,44012
month_sale,2018-05-01 00:00:00,2018-06-01 00:00:00,2018-02-01 00:00:00,2018-02-01 00:00:00,2018-02-01 00:00:00
product_ID,2335,2335,2335,2335,2335
merge_date,2018-04-01 00:00:00,2018-05-01 00:00:00,2018-01-01 00:00:00,2018-01-01 00:00:00,2018-01-01 00:00:00
partition_ID_x,2018-05-01 00:00:00,2018-06-01 00:00:00,2018-02-01 00:00:00,2018-02-01 00:00:00,2018-02-01 00:00:00
entry_channel,KHK,KHN,KHK,KAT,KHL
segment,02 - PARTICULARES,01 - TOP,01 - TOP,02 - PARTICULARES,02 - PARTICULARES
antiguedad,395.0,395.0,485.0,485.0,485.0
country_id,ES,ES,ES,ES,ES
region_code,28.0,28.0,30.0,41.0,28.0


In [197]:
df_total2.drop("partition_ID_x", axis = 1, inplace = True)
df_total2.drop("partition_ID_y", axis = 1, inplace = True)
df_total2.drop("merge_date", axis = 1, inplace = True)


In [198]:
df_total2.head().T

,0,1,2,3,4
customer_ID,33620,35063,37299,39997,44012
month_sale,2018-05-01 00:00:00,2018-06-01 00:00:00,2018-02-01 00:00:00,2018-02-01 00:00:00,2018-02-01 00:00:00
product_ID,2335,2335,2335,2335,2335
entry_channel,KHK,KHN,KHK,KAT,KHL
segment,02 - PARTICULARES,01 - TOP,01 - TOP,02 - PARTICULARES,02 - PARTICULARES
antiguedad,395.0,395.0,485.0,485.0,485.0
country_id,ES,ES,ES,ES,ES
region_code,28.0,28.0,30.0,41.0,28.0
gender,H,H,V,V,V
age,50.0,62.0,54.0,62.0,42.0


In [199]:
df_total2.isnull().sum()
# Los nulos que encontramos son nuevos clientes, que para la tabla cruzada con month-1, en el mes anterior no tenían datos

customer_ID               0
month_sale                0
product_ID                0
entry_channel             0
segment                   0
antiguedad                0
country_id                0
region_code               0
gender                    0
age                       0
salary                    0
short_term_deposit    60317
loans                 60317
mortgage              60317
funds                 60317
securities            60317
long_term_deposit     60317
credit_card           60317
payroll               60317
payroll_account       60317
emc_account           60317
debit_card            60317
em_acount             60317
dtype: int64

In [200]:
df_total2[df_total2['short_term_deposit'].isnull()]

,customer_ID,month_sale,product_ID,entry_channel,segment,antiguedad,country_id,region_code,gender,age,...,mortgage,funds,securities,long_term_deposit,credit_card,payroll,payroll_account,emc_account,debit_card,em_acount
6,100296,2018-04-01,2335,KHK,01 - TOP,395.0,ES,28.0,V,46.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,114080,2018-03-01,2335,KHL,01 - TOP,426.0,ES,28.0,V,81.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,124554,2018-10-01,2335,ZZZ,04 - NO SEGMENT,212.0,ES,8.0,H,52.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,148406,2018-02-01,2335,KHK,02 - PARTICULARES,454.0,ES,9.0,V,46.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,151135,2018-03-01,2335,KHK,01 - TOP,426.0,ES,31.0,V,55.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191851,1553436,2019-05-01,4657,ZZZ,04 - NO SEGMENT,0.0,ES,39.0,V,47.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191852,1553456,2019-05-01,4657,ZZZ,04 - NO SEGMENT,0.0,ES,28.0,H,32.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191853,1553541,2019-05-01,4657,ZZZ,04 - NO SEGMENT,0.0,ES,49.0,V,54.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191854,1553559,2019-05-01,4657,ZZZ,04 - NO SEGMENT,0.0,ES,8.0,H,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#los nulos en los productos los rellenamos con 0 porque esos cliente no debían exitir en esa fecha

In [201]:
df_total2.fillna(value = 0, inplace = True)

In [202]:
df_total2.isnull().sum()

customer_ID           0
month_sale            0
product_ID            0
entry_channel         0
segment               0
antiguedad            0
country_id            0
region_code           0
gender                0
age                   0
salary                0
short_term_deposit    0
loans                 0
mortgage              0
funds                 0
securities            0
long_term_deposit     0
credit_card           0
payroll               0
payroll_account       0
emc_account           0
debit_card            0
em_acount             0
dtype: int64

In [204]:
#df_total3=df_total2.to_pickle('/content/drive/MyDrive/Master_DS/TFM_Data_Science/df_total3')
df_total3=df_total2.to_pickle('/content/drive/MyDrive/TFM  Data Science/df_total3')

In [213]:
df_total3=pd.read_pickle('/content/drive/MyDrive/TFM  Data Science/df_total3')

In [214]:
df_total3.head().T

,0,1,2,3,4
customer_ID,33620,35063,37299,39997,44012
month_sale,2018-05-01 00:00:00,2018-06-01 00:00:00,2018-02-01 00:00:00,2018-02-01 00:00:00,2018-02-01 00:00:00
product_ID,2335,2335,2335,2335,2335
entry_channel,KHK,KHN,KHK,KAT,KHL
segment,02 - PARTICULARES,01 - TOP,01 - TOP,02 - PARTICULARES,02 - PARTICULARES
antiguedad,395.0,395.0,485.0,485.0,485.0
country_id,ES,ES,ES,ES,ES
region_code,28.0,28.0,30.0,41.0,28.0
gender,H,H,V,V,V
age,50.0,62.0,54.0,62.0,42.0


## Ordenación de columnas y tratamiento de variables del df mergeado

In [215]:
df_total3.columns.to_list()

['customer_ID',
 'month_sale',
 'product_ID',
 'entry_channel',
 'segment',
 'antiguedad',
 'country_id',
 'region_code',
 'gender',
 'age',
 'salary',
 'short_term_deposit',
 'loans',
 'mortgage',
 'funds',
 'securities',
 'long_term_deposit',
 'credit_card',
 'payroll',
 'payroll_account',
 'emc_account',
 'debit_card',
 'em_acount']

In [216]:
# Reordenación del dataset
df_total3 = df_total3.reindex(columns=[
              
              
              'month_sale',
              'product_ID',
              'customer_ID',
              'antiguedad',
              'entry_channel',
              'segment',
              'country_id',
              'region_code',
              'gender',
              'age',
              'salary',
              'short_term_deposit',
              'loans',
              'mortgage',
              'funds',
              'securities',
              'long_term_deposit',
              'credit_card',
              'payroll',
              'payroll_account',
              'emc_account',
              'debit_card',
              'em_acount',
              ])

In [217]:
df_total3.head().T

,0,1,2,3,4
month_sale,2018-05-01 00:00:00,2018-06-01 00:00:00,2018-02-01 00:00:00,2018-02-01 00:00:00,2018-02-01 00:00:00
product_ID,2335,2335,2335,2335,2335
customer_ID,33620,35063,37299,39997,44012
antiguedad,395.0,395.0,485.0,485.0,485.0
entry_channel,KHK,KHN,KHK,KAT,KHL
segment,02 - PARTICULARES,01 - TOP,01 - TOP,02 - PARTICULARES,02 - PARTICULARES
country_id,ES,ES,ES,ES,ES
region_code,28.0,28.0,30.0,41.0,28.0
gender,H,H,V,V,V
age,50.0,62.0,54.0,62.0,42.0


In [218]:
df_total3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 191856 entries, 0 to 191855
Data columns (total 23 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   month_sale          191856 non-null  datetime64[ns]
 1   product_ID          191856 non-null  object        
 2   customer_ID         191856 non-null  int64         
 3   antiguedad          191856 non-null  float64       
 4   entry_channel       191856 non-null  category      
 5   segment             191856 non-null  object        
 6   country_id          191856 non-null  object        
 7   region_code         191856 non-null  float64       
 8   gender              191856 non-null  object        
 9   age                 191856 non-null  float64       
 10  salary              191856 non-null  float64       
 11  short_term_deposit  191856 non-null  float64       
 12  loans               191856 non-null  float64       
 13  mortgage            191856 no

In [ ]:
df_total3.info()
     

# month_sale--> BORRAR
   
# product_ID --> OHE
# entry_channel --> OHE  
#country_id  --> OHE       
# segment --> OHE 
# gender --> OHE


   

In [219]:
df_pre = df_total3.copy()

In [ ]:
# sale_ID establecer como índice junto con customer_ID   
#df_pre.set_index(['sale_ID', 'customer_ID'], inplace=True)

In [ ]:
  
# month_sale --> BORRAR
# product_ID --> BORRAR 
# entry_date --> BORRAR 
# partition_date --> BORRAR       

#df_pre = df_pre.drop(['month_sale', 'product_ID', 'entry_date', 'partition_date'], axis=1)

    


In [220]:
df_pre.shape

(191856, 23)

In [ ]:
# partition_year --> A dtype INT    
# partition_month --> A dtype INT      
# partition_quarter --> A dtype INT

# region_code --> A dtype OBJECT
#df_pre[['partition_year', 'partition_month', 'partition_quarter']] = df_pre[['partition_year', 'partition_month', 'partition_quarter']].astype('int')

#df_pre['region_code']=df_pre['region_code'].astype(object)

In [ ]:
#df_pre.info()

### Transformación de categóricas

In [221]:
def OHE(dataframe, column_name):
    _dummy_dataset = pd.get_dummies(dataframe[column_name], prefix=column_name)
    dataframe = pd.concat([dataframe, _dummy_dataset], axis=1)
    return dataframe.drop(column_name, axis=1)

In [ ]:
# product_ID --> OHE
# entry_channel --> OHE  
#country_id  --> OHE       
# segment --> OHE 
# gender --> OHE

In [222]:
# OHE "product_ID"

df_pre = OHE(df_pre, "product_ID")

<ipython-input-221-e11062fb3a8e>:2: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  _dummy_dataset = pd.get_dummies(dataframe[column_name], prefix=column_name)


In [223]:
# OHE "entry_channel"

df_pre = OHE(df_pre, "entry_channel")

In [224]:
# OHE segment

df_pre = OHE(df_pre, "segment")

In [227]:
# OHE country_id 
df_pre = OHE(df_pre, "country_id")

In [228]:
# gender --> 1 = Hembras, 0 = Varones, 2 = Nulos

df_pre["gender"] = df_pre["gender"].replace("H", 1) # H es de Hembras
df_pre["gender"] = df_pre["gender"].replace("V", 0) # V es de Varones

In [ ]:
# deceased --> 1 = Fallecido, 0 = No fallecido

#df_pre["deceased"] = df_pre["deceased"].replace("N", 0) # N es no moñeco
#df_pre["deceased"] = df_pre["deceased"].replace("S", 1) # S es moñeco

In [229]:
df_pre.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 191856 entries, 0 to 191855
Data columns (total 104 columns):
 #    Column                      Dtype         
---   ------                      -----         
 0    month_sale                  datetime64[ns]
 1    customer_ID                 int64         
 2    antiguedad                  float64       
 3    gender                      int64         
 4    age                         float64       
 5    salary                      float64       
 6    short_term_deposit          float64       
 7    loans                       float64       
 8    mortgage                    float64       
 9    funds                       float64       
 10   securities                  float64       
 11   long_term_deposit           float64       
 12   credit_card                 float64       
 13   payroll                     float64       
 14   payroll_account             float64       
 15   emc_account                 float64       
 16   

In [ ]:
df_final=df_pre.to_pickle('/content/drive/MyDrive/Master_DS/TFM_Data_Science/df_preprocesado')

In [ ]:
df_final=pd.read_pickle('/content/drive/MyDrive/TFM  Data Science/df_preprocesado')

#### Correlación variables

In [ ]:
corr=df_final.corr()

In [ ]:
corr2=corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))

In [ ]:
corr2.stack()

In [ ]:
corr3=corr2.unstack().dropna()

In [ ]:
corr4 = corr3.sort_values()

In [ ]:
pd.set_option('display.max_rows',None)

In [ ]:
corr4

## Ajustamos el data set para el TARGET "pension plan"

In [ ]:
del df_final['pension_plan']

In [ ]:
del df_final['family_product_pension_plan']

In [ ]:
del df_final['net_margin']

In [ ]:
TARGET = 'product_desc_pension_plan'

In [ ]:
df_final['product_desc_pension_plan'].value_counts(normalize = True)

In [ ]:
# Un 8.04% de las ventas realizadas corresponden a planes de pensiones

In [ ]:
# Por curiosidad consultamos este dato para los TARGET1 y TARGET2 propuestos provisionales
df_final['product_desc_debit_card'].value_counts(normalize = True)

In [ ]:
df_final['product_desc_emc_account'].value_counts(normalize = True)

## Partición train/test/validation

In [ ]:
#df = df1.copy()

In [ ]:
#df.drop(["entry_date", "partition_date"],axis = 1,inplace = True)

In [ ]:
df_final.shape

In [ ]:
# La partición de validación tiene que ser lo más parecida a la realidad posible, un % del dataset siempre mayor de 1000 filas. tenemos 240.773 en total
# Tomaremos para validación el último mes registrado, mayo de 2019, y el resto de datos para train/test. Si vemos que los números descuadran mucho se propone otra división.

In [ ]:
# Partición de validación y develop
val_df = df_final[(df_final['year_sales']==2019) & (df_final['month_sales']==5)]
dev_df = df_final[~((df_final['year_sales'] == 2019) & (df_final['month_sales'] == 5))]

In [ ]:
val_df.shape # No es un mal número para validar, unos 10k registros

In [ ]:
dev_df.shape # Buen número para train-test, 230k registros

In [ ]:
val_df_y = val_df[[TARGET]]

val_df_X = val_df.drop(TARGET,axis=1)

In [ ]:
dev_df_y = dev_df[[TARGET]]

dev_df_x = dev_df.drop(TARGET,axis=1)

### Train y Test

In [ ]:
#20% del dataset de development para test

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split( dev_df_x, dev_df_y, test_size=0.2, random_state=42)

In [ ]:
#Comprobar que la distribución de ambas particiones es parecida (media, percentiles y distrribución std). Si no lo es, cambiamos el random state o utilizamos k.fold

In [ ]:
X_train.describe().T.head()

In [ ]:
X_test.describe().T.head()

# 4.- Modelling

## Decission Tree

In [ ]:
#Instanciamos el modelo
dt=DecisionTreeClassifier(max_depth=6,random_state=42, min_samples_leaf=500)

In [ ]:
#lo entrenamos con el dataset de train
dt.fit(X=X_train,y=y_train)

In [ ]:
#lo pintamos
tree_data=export_graphviz(
    decision_tree=dt,
    out_file=None, 
    max_depth=None,
    feature_names=X_test.columns,
    class_names=['no_pension_plan','pension_plan'],
    label="all",
    filled=True,
    leaves_parallel=False,
    impurity=True,
    node_ids=False,
    proportion=False,
    rotate=True,
    rounded=True,
    special_characters=False,
    precision=3,
    fontname="helvetica",
)

In [ ]:
graphviz.Source(tree_data)
# Lo que se aprecia en el arbol es algo en realidad muy intuitivo: para este TARGET lo más distintivo es el net_margin que ofrece, ya que está muy por encima del resto de productos de EasyMoney

In [ ]:
#generamos predicciones y evaluamos el modelo
y_test_pred = pd.DataFrame(dt.predict(X_test),index=y_test.index,columns=['pension_plan_prediction'])

In [ ]:
#join de y_test y y_test_pred para comparar el resultado real con la predicción
result_df=y_test.join(y_test_pred,how='inner')

In [ ]:
result_df['Success']=(result_df['product_desc_pension_plan']==result_df['pension_plan_prediction']).astype(int)

In [ ]:
result_df.head(20)

In [ ]:
#Accuracy
result_df['Success'].mean()

In [ ]:
# El modelo más tonto predicirá con un 92% de accuracy, frente al nuestro que predice con un accuracy 99.76%. NOT BAD


## Confusion matrix

In [ ]:
confusion_matrix=pd.crosstab(result_df['product_desc_pension_plan'],result_df['pension_plan_prediction'])

In [ ]:
TP=confusion_matrix.iloc[1,1]
TN=confusion_matrix.iloc[0,0]
FP=confusion_matrix.iloc[0,1]
FN=confusion_matrix.iloc[1,0]

In [ ]:
confusion_matrix
# Parece que el modelo tiende a dar más falsos negativos, pero aproximadamente está  balanceado entre FP y FN

In [ ]:
dt.score(X_test,y_test)

In [ ]:
#generamos las predicciones del dataset de test pero como probabilidades
dt.predict_proba(X_test)


In [ ]:
#seleeciono las probabilidades de #1 y lo convierto en un dataframe
y_score= pd.DataFrame(dt.predict_proba(X_test)[:,1],index=y_test.index, columns=['pension_plan_score']).sort_values(by='pension_plan_score', ascending=False)

In [ ]:
y_score

In [ ]:
# lo combino con el dataframe anterior
result_df= result_df.join(y_score)

In [ ]:
result_df.sort_values(by='pension_plan_score', ascending=False)

## F1 Score

In [ ]:
# Desarrollo matemático de F1Score
Precision=TP/ (TP+FP)
Recall= TP/(TP+FN)
f1_score=2/(1/Precision+1/Recall)

In [ ]:
# Precision = Cuántos positivos de los que detecta realmente lo son 
# Recall alta = Cuántos positivos detecta el modelo del total

In [ ]:
Precision

In [ ]:
Recall

In [ ]:
metrics.f1_score(y_test,y_test_pred)

## AUC y ROC curve

In [ ]:
metrics.roc_auc_score(result_df['product_desc_pension_plan'],result_df['pension_plan_score'])

In [ ]:
fpr,tpr,th=metrics.roc_curve(result_df['product_desc_pension_plan'],result_df['pension_plan_score'])
plt.plot(fpr,tpr)
plt.plot([0,1],[0,1],linestyle='--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC curve')
plt.show()


## Control del overfitting

In [ ]:
for i in range(1,15):
  #instancio el modelo
  dt2= DecisionTreeClassifier(max_depth=i,random_state=42)
  #entreno el modelo
  dt2.fit(X_train,y_train)
  #evaluo 
  acc_train = dt2.score(X_train,y_train)
  acc_test = dt2.score(X_test,y_test)
  print(f'Profundidad de arbol: {i}, Acc.train: {np.round(acc_train,6)},Acc.test: {np.round(acc_test,6)}')

In [ ]:
#puedo probar con diferent numero de observaciones en el nodo final
for i in range(1,15):
  dt2= DecisionTreeClassifier(max_depth=i,random_state=42,min_samples_split=500)
  dt2.fit(X_train,y_train)
  acc_train = dt2.score(X_train,y_train)
  acc_test = dt2.score(X_test,y_test)


  print(f'Profundidad de arbol: {i}, Acc.train: {np.round(acc_train,6)},Acc.test: {np.round(acc_test,6)}')

## Modelos alternativos

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#Random Forest con 10 replicas
rf = RandomForestClassifier (max_depth=11,random_state=42,min_samples_split=500,n_estimators=10)

In [ ]:
rf.fit(X_train,y_train)

In [ ]:
rf.score(X_test,y_test)
# El random forest mejora bastante le modelo, de un 99.76% de acierto a un 99.96%

In [ ]:
rf.score(val_df_X,val_df_y)

In [ ]:
# SIGUIENTES PASOS

# --> Métricas y evaluación de métricas del random forest
# --> Preparar el rebalanceo del target
# --> Acordar dos target más (me parecen buena opción la debit_card y la cuenta emc_account)
# Se me ocurre como estrategia de marketing ofrecer el pack plan de pensiones + emc_account a ambos grupos de clientes potenciales para diversificar riesgo,
# teniendo algo muy seguro y largoplazista y otra inversión más especulativa (se apoya esta estrategia en que el grueso de los clientes son gente joven a la que puede interesar el mix)
# --> Adaptar el dataset2 y dataset3 para lanzar los modelos sobre el target2 y target3

## Variables más predictivas

In [ ]:
top_features = pd.Series(rf.feature_importances_, index=X_train.columns).sort_values(ascending=False).head(20)

In [ ]:
top_features
# Aquí vemos las variables que el modelo ha considerado como más predictivas